<a href="https://colab.research.google.com/github/heyggun/ML/blob/main/12_Cross_Validation(K_fold%2C_StratifiedK_fold).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

#### K-fold 교차검증

- 일단 iris 데이터로 실습

In [3]:
from sklearn.datasets import load_iris

In [7]:
fold_iris = load_iris()
print(fold_iris.keys())

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


In [14]:
features = fold_iris.data
label = fold_iris.target

fold_df_clf = DecisionTreeClassifier()

In [11]:
Kfold = KFold(n_splits=5)
cv_accuracy = []

In [18]:
from sklearn.metrics import accuracy_score

In [21]:
n_iter = 0

for train_idx, test_idx in Kfold.split(features):

  X_train, X_test = features[train_idx], features[test_idx]
  y_train, y_test = label[train_idx], label[test_idx]

  # 학습
  fold_df_clf.fit(X_train, y_train)
  # 예측
  fold_pred = fold_df_clf.predict(X_test)

  #정확도 측정
  n_iter +=1
  accuracy = np.round(accuracy_score(y_test, fold_pred), 4)
  print(f"{n_iter}, 교차 검증 정확도 : {accuracy}, 학습 데이터 크기 : {X_train.shape[0]}, 검증 데이터 크기 : {X_test.shape[0]}")
  cv_accuracy.append(accuracy)

print(f"평균 검증 정확도: {np.mean(cv_accuracy)}")

1, 교차 검증 정확도 : 1.0, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
2, 교차 검증 정확도 : 0.9667, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
3, 교차 검증 정확도 : 0.8333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
4, 교차 검증 정확도 : 0.9333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
5, 교차 검증 정확도 : 0.7333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
평균 검증 정확도: 0.8933199999999999


< k-fold의 문제점 >

-  일정한 간격으로 잘라서 사용하기 때문에, 만약 label이 0,1,2 이지만 0,1만 가지고 학습할 경우 2라는 답을 도출할 수 없어짐

아래 예시 코드 참고



In [23]:
# 기존 Kfold 문제점

kfold_iris_data = load_iris()
kfold_iris_data_df = pd.DataFrame(data=kfold_iris_data.data, columns= kfold_iris_data.feature_names)
kfold_iris_data_df['target'] = kfold_iris_data.target

kfold_iris_data_df['target'].value_counts()

0    50
1    50
2    50
Name: target, dtype: int64

In [24]:
display(kfold_iris_data_df.head(3))

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [27]:
kfold_iris = KFold(n_splits=3)
cnt_iter= 0

for train_idx, test_inx in kfold_iris.split(kfold_iris_data_df):

  cnt_iter+=1
  label_train = kfold_iris_data_df['target'].iloc[train_idx]
  label_test = kfold_iris_data_df['target'].iloc[test_idx]

  print(f"교차검증 {cnt_iter}")
  print(f"학습 레이블 데이터 분포 \n: {label_train.value_counts()}")
  print(f"검증 레이블 데이터 분포 : {label_test.value_counts()}")

교차검증 1
학습 레이블 데이터 분포 
: 1    50
2    50
Name: target, dtype: int64
검증 레이블 데이터 분포 
: 2    30
Name: target, dtype: int64
교차검증 2
학습 레이블 데이터 분포 
: 0    50
2    50
Name: target, dtype: int64
검증 레이블 데이터 분포 
: 2    30
Name: target, dtype: int64
교차검증 3
학습 레이블 데이터 분포 
: 0    50
1    50
Name: target, dtype: int64
검증 레이블 데이터 분포 
: 2    30
Name: target, dtype: int64


## stratifiedkFold

- target에 속성값의 개수를 동일하게 가져감으로써 kfold 같이 데이터가 한곳으로 몰리는 것을 방지

In [28]:
from sklearn.model_selection import StratifiedKFold

In [40]:
skf_iris = StratifiedKFold(n_splits=3)
cnt_iter = 0

for train_idx, test_idx in skf_iris.split(kfold_iris_data_df, kfold_iris_data_df['target']):
  cnt_iter +=1
  label_train = kfold_iris_data_df['target'].iloc[train_idx]
  label_test = kfold_iris_data_df['target'].iloc[test_idx]

  print(f"교차검증 : {cnt_iter}")
  print(f"학습 레이블 데이터 분포 :\n {label_train.value_counts()} ")
  print(f"검증 레이블 데이터 분포 :\n {label_test.value_counts()}")

교차검증 : 1
학습 레이블 데이터 분포 :
 2    34
0    33
1    33
Name: target, dtype: int64 
검증 레이블 데이터 분포 :
 0    17
1    17
2    16
Name: target, dtype: int64
교차검증 : 2
학습 레이블 데이터 분포 :
 1    34
0    33
2    33
Name: target, dtype: int64 
검증 레이블 데이터 분포 :
 0    17
2    17
1    16
Name: target, dtype: int64
교차검증 : 3
학습 레이블 데이터 분포 :
 0    34
1    33
2    33
Name: target, dtype: int64 
검증 레이블 데이터 분포 :
 1    17
2    17
0    16
Name: target, dtype: int64


<stratifiedKFold 예시>
- 회귀에서는 지원하지 X

In [41]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

from sklearn.model_selection import StratifiedKFold

import pandas as pd
import numpy as np

In [47]:
iris_data = load_iris()
features = iris_data['data']
labels = iris_data['target']

clf = DecisionTreeClassifier(random_state=1234)

In [50]:
skfold = StratifiedKFold(n_splits=3)
idx_iter = 0
cv_accuracy = []


for train_idx, test_idx in skfold.split(features, labels):
  X_train, X_test = features[train_idx], features[test_idx]
  y_train, y_test = labels[train_idx], labels[test_idx]

  clf.fit(X_train, y_train)
  pred = clf.predict(X_test)

  idx_iter +=1
  accuracy = np.round(accuracy_score(y_test,pred),4)
  train_size = X_train.shape[0]
  test_size = X_test.shape[0]

  print(f"{idx_iter}, '교차 검증 정확도' : {accuracy}, '학습 데이터 크기 :' {train_size}, '검증 데이터 크기' : {test_size}")
  print(f"{idx_iter}, '검증 세트 인덱스' : {test_idx}")
  cv_accuracy.append(accuracy)


1, '교차 검증 정확도' : 0.98, '학습 데이터 크기 :' 100, '검증 데이터 크기' : 50
1, '검증 세트 인덱스' : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]
2, '교차 검증 정확도' : 0.94, '학습 데이터 크기 :' 100, '검증 데이터 크기' : 50
2, '검증 세트 인덱스' : [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]
3, '교차 검증 정확도' : 1.0, '학습 데이터 크기 :' 100, '검증 데이터 크기' : 50
3, '검증 세트 인덱스' : [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]


In [54]:
print(np.round(cv_accuracy,4))
print(np.mean(cv_accuracy))

[0.98 0.94 1.  ]
0.9733333333333333
